<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Three.js Volume Rendering with Controls</title>
    <style>
        body { margin: 0; }
        canvas { display: block; }
        #controls { position: absolute; top: 10px; left: 10px; z-index: 100; }
        #sliderControls { position: absolute; top: 50px; left: 10px; z-index: 100; }
    </style>
</head>
<body>
    <div id="controls">
        <input type="file" id="fileInput" multiple>
        <button id="startRecord">Start Recording</button>
        <button id="stopRecord" disabled>Stop Recording</button>
    </div>
    <div id="sliderControls">
        <label for="red">Red:</label>
        <input type="range" id="red" min="0" max="2" step="0.01" value="1">
        <label for="green">Green:</label>
        <input type="range" id="green" min="0" max="2" step="0.01" value="1">
        <label for="blue">Blue:</label>
        <input type="range" id="blue" min="0" max="2" step="0.01" value="1">
        <label for="brightness">Brightness:</label>
        <input type="range" id="brightness" min="0" max="2" step="0.01" value="1">
        <label for="gamma">Gamma:</label>
        <input type="range" id="gamma" min="0.1" max="3" step="0.1" value="1">
    </div>
    <script type="module" src="script.js"></script>
</body>
</html>

In [ ]:

import * as THREE from 'https://unpkg.com/three@0.153.0/build/three.module.js';
import { OrbitControls } from 'https://unpkg.com/three@0.153.0/examples/jsm/controls/OrbitControls.js';
import CCapture from 'https://cdn.jsdelivr.net/npm/ccapture.js@1.1.0/build/CCapture.all.min.js';

// Set up the scene, camera, and renderer
const scene = new THREE.Scene();
const camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
const renderer = new THREE.WebGLRenderer();
renderer.setSize(window.innerWidth, window.innerHeight);
document.body.appendChild(renderer.domElement);

// Add OrbitControls
const controls = new OrbitControls(camera, renderer.domElement);
controls.enableDamping = true;
controls.dampingFactor = 0.25;
controls.screenSpacePanning = false;
controls.maxPolarAngle = Math.PI / 2;

let texture3D, shaderMaterial, cube;
const sliceCount = 256; // Default number of slices

const uniforms = {
    uTexture: { value: null },
    uRed: { value: 1.0 },
    uGreen: { value: 1.0 },
    uBlue: { value: 1.0 },
    uBrightness: { value: 1.0 },
    uGamma: { value: 1.0 },
    uSize: { value: sliceCount }
};

// Load 2D slice images as a 3D texture
const loadSlices = (files) => {
    const sliceImages = [];
    const promises = [];
    for (const file of files) {
        const url = URL.createObjectURL(file);
        const img = new Image();
        img.src = url;
        promises.push(new Promise((resolve) => {
            img.onload = () => {
                sliceImages.push(img);
                resolve();
            };
        }));
    }

    Promise.all(promises).then(() => {
        const size = sliceImages[0].width;
        const data = new Uint8Array(size * size * sliceImages.length);

        for (let i = 0; i < sliceImages.length; i++) {
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');
            canvas.width = size;
            canvas.height = size;
            context.drawImage(sliceImages[i], 0, 0, size, size);
            const imageData = context.getImageData(0, 0, size, size).data;
            for (let j = 0; j < imageData.length; j += 4) {
                data[i * size * size + j / 4] = imageData[j]; // Use the red channel
            }
        }

        texture3D = new THREE.DataTexture3D(data, size, size, sliceImages.length);
        texture3D.format = THREE.RedFormat;
        texture3D.type = THREE.UnsignedByteType;
        texture3D.minFilter = THREE.LinearFilter;
        texture3D.magFilter = THREE.LinearFilter;
        texture3D.unpackAlignment = 1;

        uniforms.uTexture.value = texture3D;

        if (!cube) {
            createVolume();
        }
    });
};

const createVolume = () => {
    shaderMaterial = new THREE.ShaderMaterial({
        uniforms: uniforms,
        vertexShader: `
            varying vec3 vUv;
            void main() {
                vUv = position;
                gl_Position = projectionMatrix * modelViewMatrix * vec4(position, 1.0);
            }
        `,
        fragmentShader: `
            uniform sampler3D uTexture;
            uniform float uRed;
            uniform float uGreen;
            uniform float uBlue;
            uniform float uBrightness;
            uniform float uGamma;
            uniform float uSize;
            varying vec3 vUv;

            void main() {
                vec3 texCoord = vUv * uSize;
                vec4 color = texture(uTexture, texCoord / uSize);
                color.rgb = pow(color.rgb * vec3(uRed, uGreen, uBlue) * uBrightness, vec3(1.0 / uGamma));
                gl_FragColor = color;
            }
        `,
        side: THREE.BackSide
    });

    const cubeGeometry = new THREE.BoxGeometry(1, 1, 1);
    cube = new THREE.Mesh(cubeGeometry, shaderMaterial);
    scene.add(cube);
};

// Position the camera
camera.position.z = 2;

// Handle window resize
window.addEventListener('resize', () => {
    camera.aspect = window.innerWidth / window.innerHeight;
    camera.updateProjectionMatrix();
    renderer.setSize(window.innerWidth, window.innerHeight);
});

// Animation loop
let capturer;
let recording = false;

function animate() {
    requestAnimationFrame(animate);
    controls.update();
    renderer.render(scene, camera);

    if (recording && capturer) {
        capturer.capture(renderer.domElement);
    }
}

animate();

// Recording controls
document.getElementById('startRecord').addEventListener('click', () => {
    capturer = new CCapture({ format: 'webm', framerate: 60 });
    capturer.start();
    recording = true;
    document.getElementById('startRecord').disabled = true;
    document.getElementById('stopRecord').disabled = false;
});

document.getElementById('stopRecord').addEventListener('click', () => {
    recording = false;
    capturer.stop();
    capturer.save();
    document.getElementById('startRecord').disabled = false;
    document.getElementById('stopRecord').disabled = true;
});

// File input control
document.getElementById('fileInput').addEventListener('change', (event) => {
    loadSlices(event.target.files);
});

// Slider controls
document.getElementById('red').addEventListener('input', (event) => {
    uniforms.uRed.value = event.target.value;
});
document.getElementById('green').addEventListener('input', (event) => {
    uniforms.uGreen.value = event.target.value;
});
document.getElementById('blue').addEventListener('input', (event) => {
    uniforms.uBlue.value = event.target.value;
});
document.getElementById('brightness').addEventListener('input', (event) => {
    uniforms.uBrightness.value = event.target.value;
});
document.getElementById('gamma').addEventListener('input', (event) => {
    uniforms.uGamma.value = event.target.value;
});

In [ ]:
void createZXTextures(std::vector<unsigned char*>& vtImages, GLuint* textures, int numTextures) {
    // Step 1: Initialize OpenGL and CUDA
    glewInit();
    cudaGLSetGLDevice(0);
    // Step 2: Generate and bind OpenGL textures
    glGenTextures(numTextures, textures);
    for (int i = 0; i < numTextures; ++i) {
        glBindTexture(GL_TEXTURE_2D, textures[i]);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA8, WIDTH, HEIGHT, 0, GL_RGBA, GL_UNSIGNED_BYTE, NULL);

        // Step 3: Register OpenGL texture with CUDA
        cudaGraphicsResource* cudaResource;
        cudaGraphicsGLRegisterImage(&cudaResource, textures[i], GL_TEXTURE_2D, cudaGraphicsMapFlagsWriteDiscard);

        // Step 4: Map the texture resource, get the pointer to CUDA memory
        cudaArray* array;
        cudaGraphicsMapResources(1, &cudaResource, 0);
        cudaGraphicsSubResourceGetMappedArray(&array, cudaResource, 0, 0);

        // Step 5: Copy data from CPU memory to CUDA array
        cudaMemcpy2DToArray(array, 0, 0, vtImages[i], WIDTH * 4, WIDTH * 4, HEIGHT, cudaMemcpyHostToDevice);

        // Step 6: Unmap the texture resource
        cudaGraphicsUnmapResources(1, &cudaResource, 0);

        // Step 7: Unregister the resource
        cudaGraphicsUnregisterResource(cudaResource);
    }
}

void createZXTextures(std::vector<unsigned char*>& vtImages, GLuint* textures, int numTextures) {
    // Step 1: Initialize OpenGL and CUDA
    glewInit();
    cudaGLSetGLDevice(0);

    // Step 2: Generate and bind OpenGL textures
    glGenTextures(numTextures, textures);

    // Array to hold CUDA graphics resources
    cudaGraphicsResource** cudaResources = new cudaGraphicsResource*[numTextures];

    for (int i = 0; i < numTextures; ++i) {
        glBindTexture(GL_TEXTURE_2D, textures[i]);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA8, WIDTH, HEIGHT, 0, GL_RGBA, GL_UNSIGNED_BYTE, NULL);

        // Step 3: Register OpenGL texture with CUDA
        cudaGraphicsGLRegisterImage(&cudaResources[i], textures[i], GL_TEXTURE_2D, cudaGraphicsMapFlagsWriteDiscard);
        // Step 4: Map the texture resource, get the pointer to CUDA memory
        cudaArray* array;
        cudaGraphicsMapResources(1, &cudaResources[i], 0);
        cudaGraphicsSubResourceGetMappedArray(&array, cudaResources[i], 0, 0);
        // Step 5: Copy data from CPU memory to CUDA array
        cudaMemcpy2DToArray(array, 0, 0, vtImages[i], WIDTH * 4, WIDTH * 4, HEIGHT, cudaMemcpyHostToDevice);

        // Step 6: Unmap the texture resource
        cudaGraphicsUnmapResources(1, &cudaResources[i], 0);
        // Note: We do not unregister the resource here if we need to use it later.
    }

    // Cleanup: Unregister and free CUDA resources
    for (int i = 0; i < numTextures; ++i) {
        cudaGraphicsUnregisterResource(cudaResources[i]);
    }
    delete[] cudaResources;
}